In [745]:
import tensorflow as tf
import numpy as np
ep=np.exp(-10) #a small number

In [169]:
def soft_max(Qhat): #only takes float type argument
    sm=tf.nn.softmax(Qhat)
    sess=tf.Session()
    Q=sess.run(sm)
    sess.close()
    return Q

In [171]:
def one_hot_matrix(labels, K):
    """
    Creates a matrix where the i-th row corresponds to the ith dimension and the jth column
                     corresponds to the jth bin. So if dimension i had a label j, then entry (i,j) 
                     will be 1. 
                     
    Arguments:
    labels -- vector containing the bin label from function bin_b
    K -- number of bin
    """
    b = labels.flatten()
    
    # Use tf.one_hot, be careful with the axis
    one_hot_matrix = tf.one_hot(b,K,axis=1)
    sess = tf.Session()
    one_hot = sess.run(one_hot_matrix)
    sess.close()
    
    return one_hot

In [853]:
def cdf_piecewise_linear(xB,Qn): #inputs: xB, matrix Q =(dim_xB,K) computed from xA, number of bins K
    (dim,K)=np.shape(Qn)
    b=np.floor(xB*K) #get the bin number of xB (dim,1)
    #print(b)
    Qindex=np.pad([np.array(range(0,K))],((0,dim-1),(0,0)),"edge") #create a matrix outputting Q's index 
    part=(Qindex<b)*Q #output the part that is inside the sum of Eq(13), (dim,K)
    #print(cdfpart)
    proj=one_hot_matrix(b,K) #create a mask according to label b, (dim,K)
    cdftotal=np.sum(part+Q*proj*(K*xB-b),axis=1,keepdims=True) #output the cdf (dim,1)
    return cdftotal

In [854]:
def inversecdf_pw_linear(c,Qn):
    (dim,K)=np.shape(Qn)
    b = bin_b(c,Qn)
    proj=one_hot_matrix(b,K) #create a mask according to label b, (dim,K)
    Qindex=np.pad([np.array(range(0,K))],((0,dim-1),(0,0)),"edge") #create a matrix outputting Q's index 
    
    return 1/K*(np.true_divide(c-np.sum((Qindex<b)*Qn, axis=1,keepdims=True),np.sum(proj*Qn,axis=1,keepdims=True))+b)

In [ ]:
#

In [684]:
def normalize_V(V,Wn): #inputs: V =(dim_xB,K+1) and W =(dim_xB,K) computed and normalized from xA
    (dim,K)=np.shape(Wn) #get the dimension and number of bins
    expV=np.exp(V) #(dim,K+1)
    Vdeno=np.sum(0.5*(expV[:,0:K]+expV[:,1:K+1])*Wn,axis=1,keepdims=True) #(dim,1)   
    return np.true_divide(expV,Vdeno)

In [820]:
def normalize_softmax(W):
    return soft_max(W)

In [855]:
def bin_b(xB,Wn):
    #print(xB)
    #print(Wn)
    (dim,K)=np.shape(Wn)
    Wcum=np.cumsum(Wn,axis=1) #get the cdf in each dimension of W
    mask=((xB-Wcum)>ep) #to avoid the precision issue, compare to a small number epsilon instead of zero
    Wtrun=mask*Wcum 
    b=np.reshape(np.argmax(Wtrun,axis=1),(dim,1))
    b=b+np.reshape(np.amax(Wtrun,axis=1)>0,(dim,1))*np.ones((dim,1))
    return b

In [850]:
def lerp(xB,Wn,Vn):
    (dim,K)=np.shape(Wn)
    b=bin_b(xB,Wn)
    a=alpha(xB,Wn,b)
    #print(b)
    #print(a)
    proj=one_hot_matrix(b,K) #create a mask according to label b, (dim,K)
    #print(proj)
    diff= np.sum((-Vn[:,0:K]+Vn[:,1:K+1])*proj,axis=1,keepdims=True) #(dim,1)
    #print(diff)
    return a*(diff)+np.sum(Vn[:,0:K]*proj,axis=1,keepdims=True)

In [851]:
def cdf_piecewise_quad(xB,Wn,Vn):
    (dim,K)=np.shape(Wn)
    b=bin_b(xB,Wn)
    #print(b)
    a=alpha(xB,Wn,b)
    #print(a)
    proj=one_hot_matrix(b,K) #create a mask according to label b, (dim,K)
    Wb=np.reshape(np.sum(Wn*proj,axis=1),(dim,1))
    
    Vb=np.sum(Vn[:,0:K]*proj,axis=1,keepdims=True)
    Vbplusone=np.sum(Vn[:,1:K+1]*proj,axis=1,keepdims=True)

    Windex=np.pad([np.array(range(0,K))],((0,dim-1),(0,0)),"edge") #create a matrix outputting W's indices
    part=(Windex<b)*(0.5*(Vn[:,0:K]+Vn[:,1:K+1])*Wn)
    constant=np.sum(part,axis=1,keepdims=True)
    #print(constant)
    
    return (a*a*0.5*(Vbplusone-Vb)+a*Vb)*Wb+constant

In [899]:
def inversecdf_pw_quad(c,Wn,Vn):
    print(c)
    (dim,K)=np.shape(Wn)
    VW=0.5*(Vn[:,0:K]+Vn[:,1:K+1])*Wn
    print(VW)
    b=bin_b(c,VW)
    print(b)
    proj=one_hot_matrix(b,K) #create a mask according to label b, (dim,K)
    Wb=np.reshape(np.sum(Wn*proj,axis=1),(dim,1))
    Vb=np.sum(Vn[:,0:K]*proj,axis=1,keepdims=True)
    Vbplusone=np.sum(Vn[:,1:K+1]*proj,axis=1,keepdims=True)
    
    Windex=np.pad([np.array(range(0,K))],((0,dim-1),(0,0)),"edge") #create a matrix outputting W's indices   
    cprime=c-np.sum((Windex<b)*VW,axis=1,keepdims=True)
    r=np.true_divide((Vbplusone-Vb),Wb) # can be less than zero
    print(r)
    return  np.true_divide(np.sqrt(Vb*Vb+2*cprime*r)-Vb,r)+np.sum((Windex<b)*Wn,axis=1,keepdims=True)

In [916]:
#test
c=np.random.rand(4,1)
Wn1=normalize_W(np.random.rand(4,5))
Vn1=normalize_V(np.random.rand(4,6),Wn1)
xB=inversecdf_pw_quad(c,Wn1,Vn1)
cdf_piecewise_quad(xB,Wn1,Vn1)

[[0.05750805]
 [0.38194634]
 [0.69600639]
 [0.83465197]]
[[0.39635567 0.12925801 0.18587676 0.13968668 0.14882288]
 [0.20474916 0.17056116 0.1852111  0.16439688 0.27508169]
 [0.24507798 0.12356665 0.24576714 0.17920191 0.20638633]
 [0.10754031 0.1139931  0.18081527 0.42643702 0.1712143 ]]
[[0.]
 [2.]
 [3.]
 [4.]]
[[-0.96672998]
 [ 0.16332529]
 [ 1.52514034]
 [ 1.44485833]]


array([[0.05750805],
       [0.38194634],
       [0.69600639],
       [0.83465197]])